In [2]:
import pandas as pd
import numpy as np

In [5]:
data_train = pd.read_csv("train.csv")
data_val = pd.read_csv("dev.csv")
data_test = pd.read_csv("test.csv")

In [17]:
data_test.head(20)

,Source,Event,Xintent,Xemotion,Otheremotion,Xsent,Osent
0,it_events,It starts to rainy,"[""none""]","[""none""]","[""upset""]",NaN,3
1,it_events,It starts to rainy,"[""none""]","[""none""]","[""sad, gloomy""]",NaN,1
2,it_events,It starts to rainy,"[""none""]","[""none""]","[""wet"", ""gloomy""]",NaN,1
3,it_events,It is finally time,"[""none""]","[""none""]","[""excited""]",NaN,5
4,it_events,It is finally time,"[""none""]","[""none""]","[""relieved""]",NaN,5
5,it_events,It would be best,"[""none""]","[""none""]","[""rueful""]",NaN,2
6,it_events,It is a waste of time,"[""none""]","[""none""]","[""annoyed""]",NaN,1
7,it_events,It is a waste of time,"[""none""]","[""none""]","[""angry""]",NaN,2
8,it_events,It is a waste of time,"[""none""]","[""none""]","[""defeated""]",NaN,1
9,it_events,It is the best day ever,"[""none""]","[""none""]","[""happy"", ""pleased""]",NaN,5


In [45]:
Xemotion_unique = set()
for emotions in data_train["Xemotion"]:
    emotion = emotions.split(",")
    for em in emotion:
        Xemotion_unique.add(em.replace('"', "").replace("[", "").replace("]", "").replace(".", "").strip())

In [46]:
Otheremotion_unique = set()
for emotions in data_train["Otheremotion"]:
    emotion = emotions.split(",")
    for em in emotion:
        Otheremotion_unique.add(em.replace('"', "").replace("[", "").replace("]", "").replace(".", "").strip())

In [47]:
Intentions_unique = set()
for emotions in data_train["Xintent"]:
    emotion = emotions.split(",")
    for em in emotion:
        Intentions_unique.add(em.replace('"', "").replace("[", "").replace("]", "").replace(".", "").strip())

In [50]:
print(len(Intentions_unique))
print(len(Xemotion_unique))
print(len(Otheremotion_unique))

27115
10666
5618


In [52]:
Xemotion_unique

{'',
 'like a disappointment',
 'like they are relaxing',
 'tired nad weak',
 'x feels sad',
 'unexpressive',
 'heartbroken from being cheated on',
 'worried about persony',
 'needing assistance',
 'like they need more information',
 'like a chef',
 'stuffed',
 'grown',
 'satisfied because he did everything he could',
 'like this made their relationship deeper',
 'good that he can see person',
 'out there',
 'like a humorous person',
 'intense',
 'engaged & responsible',
 'excited for y to go to school',
 'relieved to have finished already',
 'impatient for it to cook',
 "proud to be a materially positive influence able to bring relief & comfort to other people's lives",
 'as though they did everything they could have done',
 'run-down',
 'bad',
 'noatalgic',
 'good about being honest',
 'tired from walking',
 'happy as they are being admired',
 'full and focused',
 'happy about having his wish fulfilled',
 'like he wants to play sports',
 'good answer',
 'goofy',
 'despondent',
 'good

In [ ]:
small_data_train = data_train.sample(n=10000)
small_data_val = data_val.sample(n = 1000)
small_data_test = data_test.sample(n = 1000)

In [ ]:
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/event2mind-2018.10.26.tar.gz")
predictor.predict(source="PersonX drops a hint")

# LAUNCH HERE

In [1]:
import sys
sys.path.insert(0, "/usr/lib/python3.7/site-packages")

from pathlib import Path
from typing import *
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
from functools import partial
from overrides import overrides

from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.nn import util as nn_util

In [2]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    testing=True,
    seed=1,
    batch_size=64,
    lr=3e-4,
    epochs=2,
    hidden_sz=64,
    max_seq_len=100, # necessary to limit memory usage
    max_vocab_size=100000,
)

In [3]:
from allennlp.common.checks import ConfigurationError

In [4]:
torch.manual_seed(config.seed)

In [5]:
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.dataset_readers import Event2MindDatasetReader

# HERE

In [6]:
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter
from allennlp.data.token_indexers import SingleIdTokenIndexer
from typing import Dict
import csv
import json
import logging

from overrides import overrides

from allennlp.common.checks import ConfigurationError
from allennlp.common.file_utils import cached_path
from allennlp.common.util import START_SYMBOL, END_SYMBOL
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.data.fields import TextField
from allennlp.data.instance import Instance
from allennlp.data.tokenizers import Token, Tokenizer
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer

In [7]:
reader = Event2MindDatasetReader()

In [95]:
train_ds, test_ds = (reader.read(fname) for fname in ["train-Copy1.csv", "test-Copy1.csv"])
val_ds = None
train_ds

4it [00:00, 200.86it/s]
4it [00:00, 217.78it/s]


# TROW EXCESS

In [96]:
# for an array of strings
def arr_hash(array):
    full_string = ""
    for element in array:
        full_string += str(element)
    return hash(full_string)%10000000

def clear(data_set):
    hashes = set()
    check_collision = {}
    i = 0
    while(i < len(data_set)):
        full_hash = arr_hash(   data_set[i].fields["source"].tokens  +
                                data_set[i].fields["xintent"].tokens +
                                data_set[i].fields["xreact"].tokens  +
                                data_set[i].fields["oreact"].tokens   )
        if(full_hash in hashes): 
            if(full_hash in check_collision.keys() and check_collision[full_hash] == data_set[i].fields["source"].tokens):
                data_set.pop(i)
            else:
                check_collision[full_hash] = data_set[i].fields["source"].tokens
        else:
            hashes.add(full_hash)
            check_collision[full_hash] = data_set[i].fields["source"].tokens
        i += 1

In [97]:
clear(train_ds)
clear(test_ds)
train_ds
#vocab = Vocabulary.from_instances(train_ds, max_vocab_size=config.max_vocab_size)

In [118]:
from allennlp.data.iterators import BucketIterator

In [122]:
iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("source", "num_tokens")],
                         )

In [123]:
iterator.index_with(vocab)

In [124]:
batch = next(iter(iterator(train_ds)))

In [125]:
batch

{'source': {'tokens': tensor([[ 2,  5,  6,  7,  8,  9, 10,  3],
          [ 2,  5,  6,  7,  8,  9, 10,  3],
          [ 2,  5,  6,  7,  8,  9, 10,  3],
          [ 2,  5,  6,  7,  8,  9, 10,  3]])},
 'xintent': {'tokens': tensor([[2, 4, 3],
          [2, 4, 3],
          [2, 4, 3],
          [2, 4, 3]])},
 'xreact': {'tokens': tensor([[2, 4, 3],
          [2, 4, 3],
          [2, 4, 3],
          [2, 4, 3]])},
 'oreact': {'tokens': tensor([[ 2, 11,  3],
          [ 2, 11,  3],
          [ 2, 12,  3],
          [ 2, 12,  3]])}}

In [126]:
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.nn.util import get_text_field_mask
from allennlp.models import Event2Mind
from allennlp.modules.text_field_embedders import TextFieldEmbedder

In [131]:
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder

token_embedding = Embedding(num_embeddings=config.max_vocab_size + 2,
                            embedding_dim=300, padding_index=0, pretrained_file="GoogleNews-vectors-negative300.bin")
# the embedder maps the input tokens to the appropriate embedding matrix
word_embeddings: TextFieldEmbedder = BasicTextFieldEmbedder({"tokens": token_embedding})

In [132]:
from allennlp.modules.seq2vec_encoders import PytorchSeq2VecWrapper
encoder: Seq2VecEncoder = PytorchSeq2VecWrapper(nn.LSTM(word_embeddings.get_output_dim(),
                                                        config.hidden_sz, bidirectional=True, batch_first=True))

In [136]:
model = Event2Mind(
    vocab=vocab,
    embedding_dropout=0.5,
    source_embedder=word_embeddings,
    max_decoding_steps=1000,
    encoder=encoder,
)

In [137]:
batch = nn_util.move_to_device(batch, -1)

In [139]:
optimizer = optim.Adam(model.parameters(), lr=config.lr)

In [140]:
from allennlp.training.trainer import Trainer

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_ds,
    cuda_device=-1,
    num_epochs=config.epochs,
)

In [141]:
metrics = trainer.train()


  0%|          | 0/1 [00:00<?, ?it/s]
loss: 2.5781 ||: 100%|██████████| 1/1 [00:59<00:00, 59.75s/it]

  0%|          | 0/1 [00:00<?, ?it/s]
loss: 2.5578 ||: 100%|██████████| 1/1 [00:26<00:00, 27.00s/it]
